# dbatools Morning Checklist

Here's a sample PowerShell notebook showing how you can use dbatools to do a quick morning check in of your SQL Server estate.

## First let's get setup
- Import the dbatools module
- Store our list of servers in a variable
  - I'm using `Get-DbaRegisteredServer` to use my registered server list, but you could also manually specify server names or use `Import-Csv` to get them from a CSV file.


In [1]:
Import-Module dbatools

$servers = Get-DbaRegisteredServer -Group Sqlserver

$servers

Name   ServerName Group     Description Source             
----   ---------- -----     ----------- ------             
mssql1 mssql1     SqlServer             Local Server Groups
mssql2 mssql2     SqlServer             Local Server Groups




## 1. Check SQL Server Backups

View last backup dates for all SQL Server databases.

In [2]:
Get-DbaLastBackup -SqlInstance $servers | 
Format-Table SqlInstance, Database, LastFullBackup, LastDiffBackup, LastLogBackup

SqlInstance Database           LastFullBackup          LastDiffBackup LastLogBackup
----------- --------           --------------          -------------- -------------
mssql1      AdventureWorks2017 2019-12-08 15:50:46.000                             
mssql1      DatabaseAdmin                                                          
mssql1      master                                                                 
mssql1      model                                                                  
mssql1      msdb                                                                   
mssql2      AdventureWorks2019                                                     
mssql2      master             2019-12-08 15:45:26.000                             
mssql2      model              2019-12-08 15:45:29.000 2019-12-08 ... 2019-12-08...
mssql2      msdb               2019-12-08 15:45:32.000 2019-12-08 ...              




You could also use the PowerShell `Where-Object` to filter out the databases that meet your backup requirements. 

In this instance:
 - Full backup within the last 7 days
 - Differential backup within the last 1 day
 - Transaction log backup within the last hour

In [3]:
Get-DbaLastBackup -SqlInstance $servers | 
Where-Object {($_.LastFullBackup.Date -lt ((Get-Date).AddDays(-7))) `
-and ($_.LastDiffBackup.Date -lt ((Get-Date).AddDays(-1))) `
-and ($_.RecoveryModel -eq 'Full' -and $_.LastLogBackup.Date -lt ((Get-Date).AddHours(-1))) } |
Format-Table SqlInstance, Database, LastFullBackup, LastDiffBackup, LastLogBackup, RecoveryModel -AutoSize

## 2. Check Last Good DBCC Checks

In [12]:
Get-DbaLastGoodCheckDb -SqlInstance $servers |
Select-Object SqlInstance, Database, DaysSinceLastGoodCheckDb, Status, DataPurityEnabled | 
Where-Object Status -ne 'OK' |
Format-Table    

## 3. Check for Failed Jobs

In [13]:
Get-DbaAgentJob -SqlInstance $servers -ExcludeDisabledJobs | 
Where-Object LastRunOutcome -ne 'Succeeded' | 
Format-Table SqlInstance, Name, LastRunDate, LastRunOutcome

## 4. Check SQL Server Error Log Messages

In [14]:
Get-DbaErrorLog -SqlInstance $servers -After (get-date).AddDays(-1) |
Format-Table SqlInstance, SqlInstance, Source, Text

In [16]:
Get-DbaErrorLog -SqlInstance $servers -After (get-date).AddDays(-1) -Text "Login Failed" |
Format-Table SqlInstance, SqlInstance, Source, Text

## 5. Add your custom checks

Perhaps in your environment you have additional things to check

- **Get-DbaAvailabilityGroup** - Check your availability groups and determine which node is primary 
- **Get-DbaDiskSpace** - Check the available disk space on your servers and ensure you're not close to running out
- **Test-DbaDbOwner** - Make sure all your databases are owned by sa, or specify an owner with the `-TargetLogin` Parameter
- **Test-DbaDbRecoveryModel** - Check your database recovery models are as expected, including not in pseudo simple mode
- **Test-DbaRepLatency** - Test replication latency with tracer tokens
